In [1]:
import kpacker
import storage
import copy
import py3dbp
import numpy as np



# def main(items=None):

    
def get_kitem():
    si = storage.Item()
    dims = si.dimensions
    return kpacker.Item('item', dims[0], dims[1], dims[2], 10)



ITEMS = []
for i in range(100):
    ITEMS.append(get_kitem())
    
    
    
def get_kbc():
    bc = storage.Struct(**storage.BulkContainer().dimensions['inner'])
    return py3dbp.Bin('bc', bc.length, bc.width, bc.height, 5000)


def get_kbin():
    binn = storage.Struct(**storage.Bin().dimensions['inner'])
    return py3dbp.Bin('bin', binn.length, binn.width, binn.height, 5000)

def get_ktote():
    tote = storage.Struct(**storage.Tote().dimensions['inner'])
    return py3dbp.Bin('tote', tote.length, tote.width, tote.height, 5000)





def get_bin(btype=None):
    if btype in ('small','s'):
        return get_kbin()
    elif btype in ('medium','m'):
        return get_ktote()
    else:
        return get_kbc()

def get_bins(bin_combo):
    
    bins_output = []
    
    num_binn = bin_combo[0]
    num_tote = bin_combo[1]
    num_bc   = bin_combo[2]
    
    for n in range(num_binn):
        bins_output.append(get_kbin())
        
    for n in range(num_tote):
        bins_output.append(get_ktote())
        
    for n in range(num_bc):
        bins_output.append(get_kbc())
        
    return bins_output

def enrich(packer):
    '''
    f: packer -> packer
    
    set properites volume, item_volume,
    filling_ratio, and empty for each bin
    
    set wasted volume for packer
    '''
    
    for b in packer.bins:
        # bin_vol = float(b.get_volume())
        b.volume = float(b.get_volume())
        b.item_vol = 0

        for item in b.items:
            item.volume = float(item.get_volume())
            b.item_vol += item.volume

        b.filling_ratio = b.item_vol/b.volume
        b.empty = b.volume - b.item_vol

    packer.wasted_volume = sum([b.empty for b in packer.bins])
    packer.n_unfit = len(packer.items)
    packer.packed = packer.n_unfit == 0
    packer.success = False or packer.packed
    
    return packer

# print(packer.wasted_volume)


def drop_empty_bins(packer):
    good_bins = []

    for idx, b in enumerate(packer.bins):
        if not(len(b.items)) == 0:
            print(idx, len(b.items))
            good_bins.append(b)

    packer.bins = good_bins
    packer = enrich(packer)
    return packer


def report(packer):
    try:  print(f"PACKER BIN COMBO: {packer.combo}")
    except: pass
    print(f"PACKER WASTED VOLUME:\t{packer.wasted_volume}")
    print(f"SUCCESS: {packer.packed}")
    print(f"PACKER UNFIT ITEMS: \t{len(packer.items)}\n")

    for b in packer.bins:
        print("\t:::::::::::", b.string())

        print(f"\tFITTED ITEMS:\t{len(b.items)}")
        # for item in b.items:
        #     print("====> ", item.string())

        print(f"\tUNFITTED ITEMS:\t{len(b.unfitted_items)}")
        # for item in b.unfitted_items:
        #     print("====> ", item.string())

        print(f"\tFILLING RATIO:\t{b.filling_ratio}")

        print("\t***************************************************")
        # print("***************************************************")
        
        
packer = py3dbp.Packer()


# ITEMS = []
# for i in range(100):
#     ITEMS.append(get_kitem())

for item in ITEMS:
    packer.add_item(item)

In [2]:

def get_combos(N):
    return  [ (x,y,z) for x in range(N) for y in range(N) for z in range(N) ]

# print(get_combos(3))





all_results = dict()
# good_results = dict()
# results = dict()
wasted_volumes = dict()

for bin_combo in get_combos(2):


    packer = py3dbp.Packer()
    packer.combo = bin_combo

    for b in get_bins(bin_combo):
        packer.add_bin(b)

    for item in ITEMS:
        packer.add_item(item)


    packer.pack(distribute_items=True)
    packer = enrich(packer)
#     report(packer)
    
    
    all_results[bin_combo] = packer
    # if packer.success == True:
    #     results[bin_combo] = packer
        
    wasted_volumes[bin_combo] = packer.wasted_volume
    


def find_best_result(results):

    best_vol = np.inf
    best_k = None

    for k,v in results.items():
        if v.packed == True:
            if v.wasted_volume < best_vol:
                best_vol = v.wasted_volume
                best_k = k

    return best_k


print(find_best_result(all_results))

(0, 0, 1)


In [ ]:
packer = py3dbp.Packer()
# packer.add_bin(get_kbin())
# packer.add_bin(get_ktote())
bin_combo = (0,1,0)
BINS = get_bins(bin_combo)

for b in BINS:
    packer.add_bin(b)

for item in ITEMS:
    packer.add_item(item)
    
packer.pack(distribute_items=1)

packer = enrich(packer)
report(packer)